In [ ]:
# ⚙️ GUI setup with widgets, decay slider, and mode selection (already provided above)

In [ ]:

# ▶️ Additional analysis: display Max Sharpe and Min Volatility portfolio results

from pypfopt.efficient_frontier import EfficientFrontier

try:
    df = df_global[selected_assets]
    logret = log_returns(df).astype(np.float64)
    decay = decay_slider.value
    cov = ewma_covariance_matrix(logret.values, alpha=decay)
    cov = pd.DataFrame(cov * 50, index=df.columns, columns=df.columns)
    mu = {asset: float(expected_return_inputs[asset].value.strip() or 0) for asset in selected_assets}
    mu = pd.Series(mu)

    ef_sharpe = EfficientFrontier(mu, cov, weight_bounds=(0, 1))
    ef_sharpe.add_objective(objective_functions.L2_reg, gamma=gamma_slider.value)
    ef_sharpe.add_sector_constraints(sector_mapper,
                                     {s: pair[0].value for s, pair in sector_constraint_widgets.items()},
                                     {s: pair[1].value for s, pair in sector_constraint_widgets.items()})
    ef_sharpe.max_sharpe()
    sharpe_weights = ef_sharpe.clean_weights()
    sharpe_perf = ef_sharpe.portfolio_performance()
    print("📈 Max Sharpe Portfolio:")
    print("Weights:", sharpe_weights)
    print("Performance (Return, Volatility, Sharpe):", sharpe_perf)

    ef_minvol = EfficientFrontier(mu, cov, weight_bounds=(0, 1))
    ef_minvol.add_objective(objective_functions.L2_reg, gamma=gamma_slider.value)
    ef_minvol.add_sector_constraints(sector_mapper,
                                     {s: pair[0].value for s, pair in sector_constraint_widgets.items()},
                                     {s: pair[1].value for s, pair in sector_constraint_widgets.items()})
    ef_minvol.min_volatility()
    minvol_weights = ef_minvol.clean_weights()
    minvol_perf = ef_minvol.portfolio_performance()
    print("\n🛡 Min Volatility Portfolio:")
    print("Weights:", minvol_weights)
    print("Performance (Return, Volatility, Sharpe):", minvol_perf)

except Exception as e:
    print("❌ Error during additional analysis:", e)
